In [35]:
import pandas as pd 
import numpy as np
import re 

In [37]:
df = pd.read_excel('/Users/nyagaderrick/Developer/Report_automation/data/Feb/output.xlsx')

In [38]:
df.head()

,Date,Names,Clock In,Clock Out
0,01 Feb 2024,JacklineOira,08:05:00,18:31:00
1,NaN,KennedyMungai,07:57:00,19:32:00
2,NaN,RhoneKwamboka,08:36:00,18:17:00
3,NaN,HumphreyMagawi,07:54:00,18:21:00
4,NaN,CeciliaMugure,_,18:21:00


In [40]:
# Fill NaN in 'Date' column with forward fill method
df['Date'].fillna(method='ffill', inplace=True)

# Convert 'Clock In' to datetime.time
df['Clock In'] = pd.to_datetime(df['Clock In'], format='%H:%M:%S', errors='coerce').dt.time

# Define the time for comparison
time_for_comparison = pd.to_datetime('08:30:00', format='%H:%M:%S').time()

# Create a new DataFrame for summary
summary = pd.DataFrame(df['Names'].unique(), columns=['Names'])
# Count the number of times each person clocked in after 8:30
late_clock_in = df[df['Clock In'] > time_for_comparison].groupby('Names')['Clock In'].count()
late_clock_in = late_clock_in.reindex(summary['Names']).fillna(0)
summary['Clock in after 8:30'] = late_clock_in.values

# Count the number of times they didn't clock in or out
no_clock_in_out = df[(df['Clock In'].isna()) | (df['Clock In'] == '_') | (df['Clock Out'].isna()) | (df['Clock Out'] == '_')].groupby('Names').size()
no_clock_in_out = no_clock_in_out.reindex(summary['Names']).fillna(0)
summary['No clock in/out'] = no_clock_in_out.values

# Convert the counts to integers
summary = summary.astype({'Clock in after 8:30': 'int32', 'No clock in/out': 'int32'})

In [41]:
summary

,Names,Clock in after 8:30,No clock in/out
0,JacklineOira,0,11
1,KennedyMungai,0,5
2,RhoneKwamboka,12,13
3,HumphreyMagawi,1,11
4,CeciliaMugure,3,17
5,FaithChepkirui,0,5
6,NancyMacharia,0,6
7,HidayaSaidi,2,14
8,JimnahNjue,0,10
9,BerniceKata,3,16


In [44]:
#add summary to the excel file after the last date 
with pd.ExcelWriter('/Users/nyagaderrick/Developer/Report_automation/data/Feb/output.xlsx', mode='a') as writer:
    summary.to_excel(writer, sheet_name='Sheet2', startrow=0, startcol=3, index=False)
